# 0 Imports

In [1]:
import numpy as np
import matplotlib as pl
import seaborn as sns
import pandas as pd

import re

from bs4 import BeautifulSoup as bs
import nltk
from nltk import pos_tag
from nltk.corpus import stopwords, wordnet
from nltk.stem import SnowballStemmer, WordNetLemmatizer
# nltk.download()

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

np.random.seed(0)


***
# 1 Config

In [26]:
config = {
    "preproc": False,
    "processing": True
}

***
# 2 Data preprocessing

## 2.1 Loading

In [3]:
if config["preproc"]:
    data = pd.read_csv("data/stackoverflow_posts_1m.csv")
    data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 4 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   Id      1000000 non-null  int64 
 1   Title   422825 non-null   object
 2   Body    999558 non-null   object
 3   Tags    422825 non-null   object
dtypes: int64(1), object(3)
memory usage: 30.5+ MB


In [4]:
display = None
if config["preproc"]:
    display = data.head()
display

,Id,Title,Body,Tags
0,525063,Android Respond To URL in Intent,<p>I want my intent to be launched when the user goes to a certain url: for example the android market does this with <a href= http://market.android.com/ >http://market.android.com/</a> urls. so does youtube. I want mine to do that too.</p>,android|url|android-intent|intentfilter|launch
1,14128723,Eclipse -- Progress windows don't show up any more,<p>I used to see both Building and Cleaning actions in their own separate windows where I could wait for them to complete before continuing. Several weeks ago I must have clicked Run in Background because now the progress only appears in a small obscured Progress view.</p> <p><img src= http://i.stack.imgur.com/Ik1x8.gif alt= Not so nice. ></p> <p>How do I set the progress of things like Java Build and Clean to appear in a popup dialog again?</p> <p><img src= http://i.stack.imgur.com/PtQsk.gif alt= Nice. ></p>,java|eclipse
2,20728,What's the best way to create ClickOnce deployments,<p>Our team develops distributed winform apps. We use ClickOnce for deployment and are very pleased with it.</p> <p>However we've found the pain point with ClickOnce is in <em>creating</em> the deployments. We have the standard dev/test/production environments and need to be able to create deployments for each of these that install and update separate from one another. Also we want control over what assemblies get deployed. Just because an assembly was compiled doesn't mean we want it deployed.</p> <p>The obvious first choice for creating deployments is Visual Studio. However VS really doesn't address the issues stated. The next in line is the SDK tool Mage. Mage works OK but creating deployments is rather tedious and we don't want every developer having our code signing certificate and password.</p> <p>What we ended up doing was rolling our own deployment app that uses the command line version of Mage to create the ClickOnce manifest files.</p> <p>I'm satisfied with our current solution but is seems like there would be an industry-wide accepted approach to this problem. Is there?</p>,winforms|deployment|clickonce
3,6531409,iOS: how to get image dimensions without opening it,<p>In an iOS app I need to provide image filters based on their size (width/height) think of something similar to Large Medium Small in Google images search. Opening each image and reading its dimensions when creating the list would be very performance intensive. Is there a way to get this info without opening the image itself?</p> <p>Damien DeVille answered the question below based on his suggestion I am now using the following code:</p> <pre><code>NSURL *imageURL = [NSURL fileURLWithPath:imagePath]; if (imageURL == nil) return; CGImageSourceRef imageSourceRef = CGImageSourceCreateWithURL((CFURLRef)imageURL NULL); if(imageSourceRef == NULL) return; CFDictionaryRef props = CGImageSourceCopyPropertiesAtIndex(imageSourceRef 0 NULL); CFRelease(imageSourceRef); NSLog(@ %@ (NSDictionary *)props); CFRelease(props); </code></pre>,ios4|uikit
4,593205,action delegate with zero parameters,<p>I see this line in many online examples of using the Action delegate:</p> <pre><code>public event Action MyEvent; </code></pre> <p>But when I try it in my own code I get this error</p> <blockquote> <p>Using the generic type 'System.Action' requires '1' type arguments</p> </blockquote> <p>The documentation certainly describes a form of Action without any type parameter. What am I missing?</p>,c#|delegates|action


In [5]:
if config["preproc"]:
    data = data.dropna(subset=["Tags"])
    data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 422825 entries, 0 to 999999
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   Id      422825 non-null  int64 
 1   Title   422825 non-null  object
 2   Body    422825 non-null  object
 3   Tags    422825 non-null  object
dtypes: int64(1), object(3)
memory usage: 16.1+ MB


***
## 2.2 Html parsing

In [6]:
if config["preproc"]:
    def preproc_raw_data_body(cell):
        soup = bs(cell, "html.parser")

        script_tags = soup.find_all("script")
        for script_tag in script_tags:
            script_tag.extract()

        code_tags = soup.find_all("code")
        for code_tag in code_tags:
            code_tag.extract()

        preproc_cell = soup.get_text()
        preproc_cell = preproc_cell.replace(',', ' ')

        return preproc_cell

In [7]:
if config["preproc"]:
    data["Body"] = data.apply(lambda row: preproc_raw_data_body(row.Body), axis=1)

***
## 2.3 Tags to list

In [8]:
if config["preproc"]:
    def tags_to_list(cell):
        return [tag for tag in re.split(r'[|]', cell) if tag]

In [9]:
if config["preproc"]:
    data["Tags"] = data.apply(lambda row: tags_to_list(row["Tags"]), axis="columns")

In [10]:
display = None
if config["preproc"]:
    display = data[["Tags"]].head()
display

,Tags
0,"[android, url, android-intent, intentfilter, launch]"
1,"[java, eclipse]"
2,"[winforms, deployment, clickonce]"
3,"[ios4, uikit]"
4,"[c#, delegates, action]"


***
## 2.4 Lowering

In [11]:
if config["preproc"]:
    def lower(cell):
        return cell.lower()

In [12]:
if config["preproc"]:
    data["Title"] = data.apply(lambda row: lower(row["Title"]), axis="columns")
    data["Body"] = data.apply(lambda row: lower(row["Body"]), axis="columns")

In [13]:
display = None
if config["preproc"]:
    display = data[["Title", "Body"]].head()
display

,Title,Body
0,android respond to url in intent,i want my intent to be launched when the user goes to a certain url: for example the android market does this with http://market.android.com/ urls. so does youtube. i want mine to do that too.
1,eclipse -- progress windows don't show up any more,i used to see both building and cleaning actions in their own separate windows where i could wait for them to complete before continuing. several weeks ago i must have clicked run in background because now the progress only appears in a small obscured progress view. how do i set the progress of things like java build and clean to appear in a popup dialog again?
2,what's the best way to create clickonce deployments,our team develops distributed winform apps. we use clickonce for deployment and are very pleased with it. however we've found the pain point with clickonce is in creating the deployments. we have the standard dev/test/production environments and need to be able to create deployments for each of these that install and update separate from one another. also we want control over what assemblies get deployed. just because an assembly was compiled doesn't mean we want it deployed. the obvious first choice for creating deployments is visual studio. however vs really doesn't address the issues stated. the next in line is the sdk tool mage. mage works ok but creating deployments is rather tedious and we don't want every developer having our code signing certificate and password. what we ended up doing was rolling our own deployment app that uses the command line version of mage to create the clickonce manifest files. i'm satisfied with our current solution but is seems like there would be an industry-wide accepted approach to this problem. is there?
3,ios: how to get image dimensions without opening it,in an ios app i need to provide image filters based on their size (width/height) think of something similar to large medium small in google images search. opening each image and reading its dimensions when creating the list would be very performance intensive. is there a way to get this info without opening the image itself? damien deville answered the question below based on his suggestion i am now using the following code:
4,action delegate with zero parameters,i see this line in many online examples of using the action delegate: but when i try it in my own code i get this error using the generic type 'system.action' requires '1' type arguments the documentation certainly describes a form of action without any type parameter. what am i missing?


***
## 2.5 Save preprocessed data

In [14]:
display = None
if config["preproc"]:
    display = data.head()
display

,Id,Title,Body,Tags
0,525063,android respond to url in intent,i want my intent to be launched when the user goes to a certain url: for example the android market does this with http://market.android.com/ urls. so does youtube. i want mine to do that too.,"[android, url, android-intent, intentfilter, launch]"
1,14128723,eclipse -- progress windows don't show up any more,i used to see both building and cleaning actions in their own separate windows where i could wait for them to complete before continuing. several weeks ago i must have clicked run in background because now the progress only appears in a small obscured progress view. how do i set the progress of things like java build and clean to appear in a popup dialog again?,"[java, eclipse]"
2,20728,what's the best way to create clickonce deployments,our team develops distributed winform apps. we use clickonce for deployment and are very pleased with it. however we've found the pain point with clickonce is in creating the deployments. we have the standard dev/test/production environments and need to be able to create deployments for each of these that install and update separate from one another. also we want control over what assemblies get deployed. just because an assembly was compiled doesn't mean we want it deployed. the obvious first choice for creating deployments is visual studio. however vs really doesn't address the issues stated. the next in line is the sdk tool mage. mage works ok but creating deployments is rather tedious and we don't want every developer having our code signing certificate and password. what we ended up doing was rolling our own deployment app that uses the command line version of mage to create the clickonce manifest files. i'm satisfied with our current solution but is seems like there would be an industry-wide accepted approach to this problem. is there?,"[winforms, deployment, clickonce]"
3,6531409,ios: how to get image dimensions without opening it,in an ios app i need to provide image filters based on their size (width/height) think of something similar to large medium small in google images search. opening each image and reading its dimensions when creating the list would be very performance intensive. is there a way to get this info without opening the image itself? damien deville answered the question below based on his suggestion i am now using the following code:,"[ios4, uikit]"
4,593205,action delegate with zero parameters,i see this line in many online examples of using the action delegate: but when i try it in my own code i get this error using the generic type 'system.action' requires '1' type arguments the documentation certainly describes a form of action without any type parameter. what am i missing?,"[c#, delegates, action]"


In [15]:
if config["preproc"]:
    data.to_csv("data/stackoverflow_posts_preprocessed.csv", index=False)

***
# 3 Data preprocessing

## 3.0 Utils

In [18]:
if config["processing"]:
    def tokenize(*texts):
        tokens = []
        for text in texts:
            # https://regex101.com/
            tokenizer = nltk.RegexpTokenizer(r'\.?[a-z#]+')
            tokens_temp = tokenizer.tokenize(text)
            tokens += [re.sub("(.)\\1{3,}", "\\1", token) for token in tokens_temp]
        return tokens

In [19]:
if config["processing"]:
    def remove_stop_words(cell):
        return [word for word in cell if word not in stop_words]

In [20]:
if config["processing"]:
    def get_wordnet_pos(treebank_tag):
        if treebank_tag.startswith('J'):
            return wordnet.ADJ
        elif treebank_tag.startswith('V'):
            return wordnet.VERB
        elif treebank_tag.startswith('N'):
            return wordnet.NOUN
        elif treebank_tag.startswith('R'):
            return wordnet.ADV
        else:
            return 'n'

In [21]:
if config["processing"]:
    def tag_pos(cell):
        treebank_tags = pos_tag(cell)
        pos = [(tag[0], get_wordnet_pos(tag[1])) for tag in treebank_tags]
        return pos

In [22]:
if config["processing"]:
    def lemmatize(cell, with_pos=False):
        lemmatizer = WordNetLemmatizer()
        if not with_pos:
            lemmatized_tokens = [lemmatizer.lemmatize(token) for token in cell]
        else:
            lemmatized_tokens = [lemmatizer.lemmatize(pair[0], pos=pair[1]) for pair in cell]
        return lemmatized_tokens

In [23]:
if config["processing"]:
    def stemmize(cell):
        stemmer = SnowballStemmer("english")
        stemmed_tokens = [stemmer.stem(token) for token in cell]
        return stemmed_tokens

***
## 3.1 Loading data

In [24]:
if config["processing"]:
    data = pd.read_csv("data/stackoverflow_posts_preprocessed.csv", index_col="Id")
    data.info()

/opt/anaconda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


<class 'pandas.core.frame.DataFrame'>
Index: 570825 entries, 525063 to 19839956
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   Title   570825 non-null  object
 1   Body    570603 non-null  object
 2   Tags    422822 non-null  object
dtypes: object(3)
memory usage: 17.4+ MB


In [41]:
if config["processing"]:
    data = data.dropna()
    data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 422603 entries, 525063 to 19839956
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   Title   422603 non-null  object
 1   Body    422603 non-null  object
 2   Tags    422603 non-null  object
dtypes: object(3)
memory usage: 12.9+ MB


In [40]:
display = None
if config["processing"]:
    display = data.head()
display

,Title,Body,Tags
Id,,,
525063,android respond to url in intent,i want my intent to be launched when the user goes to a certain url: for example the android market does this with http://market.android.com/ urls. so does youtube. i want mine to do that too.,"['android', 'url', 'android-intent', 'intentfilter', 'launch']"
14128723,eclipse -- progress windows don't show up any more,i used to see both building and cleaning actions in their own separate windows where i could wait for them to complete before continuing. several weeks ago i must have clicked run in background because now the progress only appears in a small obscured progress view. how do i set the progress of things like java build and clean to appear in a popup dialog again?,"['java', 'eclipse']"
20728,what's the best way to create clickonce deployments,our team develops distributed winform apps. we use clickonce for deployment and are very pleased with it. however we've found the pain point with clickonce is in creating the deployments. we have the standard dev/test/production environments and need to be able to create deployments for each of these that install and update separate from one another. also we want control over what assemblies get deployed. just because an assembly was compiled doesn't mean we want it deployed. the obvious first choice for creating deployments is visual studio. however vs really doesn't address the issues stated. the next in line is the sdk tool mage. mage works ok but creating deployments is rather tedious and we don't want every developer having our code signing certificate and password. what we ended up doing was rolling our own deployment app that uses the command line version of mage to create the clickonce manifest files. i'm satisfied with our current solution but is seems like there would be an industry-wide accepted approach to this problem. is there?,"['winforms', 'deployment', 'clickonce']"
6531409,ios: how to get image dimensions without opening it,in an ios app i need to provide image filters based on their size (width/height) think of something similar to large medium small in google images search. opening each image and reading its dimensions when creating the list would be very performance intensive. is there a way to get this info without opening the image itself? damien deville answered the question below based on his suggestion i am now using the following code:,"['ios4', 'uikit']"
593205,action delegate with zero parameters,i see this line in many online examples of using the action delegate: but when i try it in my own code i get this error using the generic type 'system.action' requires '1' type arguments the documentation certainly describes a form of action without any type parameter. what am i missing?,"['c#', 'delegates', 'action']"


***
## 3.2 Creating unique corpus

In [27]:
if config["processing"]:
    unique_corpus_body = " ".join([str(item) for item in data["Body"].values])
    unique_corpus_title = " ".join([str(item) for item in data["Title"].values])
    unique_corpus = unique_corpus_body + unique_corpus_title
    unique_corpus = tokenize(unique_corpus)

In [28]:
display = None
if config["processing"]:
    display = unique_corpus[:10]
display

['i', 'want', 'my', 'intent', 'to', 'be', 'launched', 'when', 'the', 'user']

***
## 3.2 Tokenize

In [44]:
if config["processing"]:
    data["Tokens"] = data.apply(lambda row: tokenize(row["Body"] + row["Title"]), axis="columns")

In [45]:
display = None
if config["processing"]:
    display = data[["Title", "Body", "Tokens"]].head()
display

,Title,Body,Tokens
Id,,,
525063,android respond to url in intent,i want my intent to be launched when the user goes to a certain url: for example the android market does this with http://market.android.com/ urls. so does youtube. i want mine to do that too.,"[i, want, my, intent, to, be, launched, when, the, user, goes, to, a, certain, url, for, example, the, android, market, does, this, with, http, market, .android, .com, urls, so, does, youtube, i, want, mine, to, do, that, too, .android, respond, to, url, in, intent]"
14128723,eclipse -- progress windows don't show up any more,i used to see both building and cleaning actions in their own separate windows where i could wait for them to complete before continuing. several weeks ago i must have clicked run in background because now the progress only appears in a small obscured progress view. how do i set the progress of things like java build and clean to appear in a popup dialog again?,"[i, used, to, see, both, building, and, cleaning, actions, in, their, own, separate, windows, where, i, could, wait, for, them, to, complete, before, continuing, several, weeks, ago, i, must, have, clicked, run, in, background, because, now, the, progress, only, appears, in, a, small, obscured, progress, view, how, do, i, set, the, progress, of, things, like, java, build, and, clean, to, appear, in, a, popup, dialog, again, eclipse, progress, windows, don, t, show, up, any, more]"
20728,what's the best way to create clickonce deployments,our team develops distributed winform apps. we use clickonce for deployment and are very pleased with it. however we've found the pain point with clickonce is in creating the deployments. we have the standard dev/test/production environments and need to be able to create deployments for each of these that install and update separate from one another. also we want control over what assemblies get deployed. just because an assembly was compiled doesn't mean we want it deployed. the obvious first choice for creating deployments is visual studio. however vs really doesn't address the issues stated. the next in line is the sdk tool mage. mage works ok but creating deployments is rather tedious and we don't want every developer having our code signing certificate and password. what we ended up doing was rolling our own deployment app that uses the command line version of mage to create the clickonce manifest files. i'm satisfied with our current solution but is seems like there would be an industry-wide accepted approach to this problem. is there?,"[our, team, develops, distributed, winform, apps, we, use, clickonce, for, deployment, and, are, very, pleased, with, it, however, we, ve, found, the, pain, point, with, clickonce, is, in, creating, the, deployments, we, have, the, standard, dev, test, production, environments, and, need, to, be, able, to, create, deployments, for, each, of, these, that, install, and, update, separate, from, one, another, also, we, want, control, over, what, assemblies, get, deployed, just, because, an, assembly, was, compiled, doesn, t, mean, we, want, it, deployed, the, obvious, first, choice, for, creating, deployments, is, visual, studio, however, vs, really, doesn, t, address, the, issues, stated, ...]"
6531409,ios: how to get image dimensions without opening it,in an ios app i need to provide image filters based on their size (width/height) think of something similar to large medium small in google images search. opening each image and reading its dimensions when creating the list would be very performance intensive. is there a way to get this info without opening the image itself? damien deville answered the question below based on his suggestion i am now using the following code:,"[in, an, ios, app, i, need, to, provide, image, filters, based, on, their, size, width, height, think, of, something, similar, to, large, medium, small, in, google, images, search, opening, each, image, and, reading, its, dimensions, when, creating, the,

***
## 3.3 StopWords deletion

In [46]:
if config["processing"]:
    stop_words = stopwords.words("english")
    print(f"stopwords: {stop_words}")

stopwords: ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so'

In [47]:
if config["processing"]:
    data["Tokens"] = data.apply(lambda row: remove_stop_words(row["Tokens"]), axis="columns")

In [48]:
display = None
if config["processing"]:
    display = data[["Title", "Body", "Tokens"]].head()
display

,Title,Body,Tokens
Id,,,
525063,android respond to url in intent,i want my intent to be launched when the user goes to a certain url: for example the android market does this with http://market.android.com/ urls. so does youtube. i want mine to do that too.,"[want, intent, launched, user, goes, certain, url, example, android, market, http, market, .android, .com, urls, youtube, want, mine, .android, respond, url, intent]"
14128723,eclipse -- progress windows don't show up any more,i used to see both building and cleaning actions in their own separate windows where i could wait for them to complete before continuing. several weeks ago i must have clicked run in background because now the progress only appears in a small obscured progress view. how do i set the progress of things like java build and clean to appear in a popup dialog again?,"[used, see, building, cleaning, actions, separate, windows, could, wait, complete, continuing, several, weeks, ago, must, clicked, run, background, progress, appears, small, obscured, progress, view, set, progress, things, like, java, build, clean, appear, popup, dialog, eclipse, progress, windows, show]"
20728,what's the best way to create clickonce deployments,our team develops distributed winform apps. we use clickonce for deployment and are very pleased with it. however we've found the pain point with clickonce is in creating the deployments. we have the standard dev/test/production environments and need to be able to create deployments for each of these that install and update separate from one another. also we want control over what assemblies get deployed. just because an assembly was compiled doesn't mean we want it deployed. the obvious first choice for creating deployments is visual studio. however vs really doesn't address the issues stated. the next in line is the sdk tool mage. mage works ok but creating deployments is rather tedious and we don't want every developer having our code signing certificate and password. what we ended up doing was rolling our own deployment app that uses the command line version of mage to create the clickonce manifest files. i'm satisfied with our current solution but is seems like there would be an industry-wide accepted approach to this problem. is there?,"[team, develops, distributed, winform, apps, use, clickonce, deployment, pleased, however, found, pain, point, clickonce, creating, deployments, standard, dev, test, production, environments, need, able, create, deployments, install, update, separate, one, another, also, want, control, assemblies, get, deployed, assembly, compiled, mean, want, deployed, obvious, first, choice, creating, deployments, visual, studio, however, vs, really, address, issues, stated, next, line, sdk, tool, mage, mage, works, ok, creating, deployments, rather, tedious, want, every, developer, code, signing, certificate, password, ended, rolling, deployment, app, uses, command, line, version, mage, create, clickonce, manifest, files, satisfied, current, solution, seems, like, would, industry, wide, accepted, approach, problem, best, way, create, ...]"
6531409,ios: how to get image dimensions without opening it,in an ios app i need to provide image filters based on their size (width/height) think of something similar to large medium small in google images search. opening each image and reading its dimensions when creating the list would be very performance intensive. is there a way to get this info without opening the image itself? damien deville answered the question below based on his suggestion i am now using the following code:,"[ios, app, need, provide, image, filters, based, size, width, height, think, something, similar, large, medium, small, google, images, search, opening, image, reading, dimensions, creating, list, would, performance, intensive, way, get, info, without, opening, image, damien, deville, answered, question, based, suggestion, using, following, code, ios, get, image, dimensions, without, opening]"
5

***
## 3.4 POS - Part-Of-Speech

In [49]:
if config["processing"]:
    data["POS"] = data.apply(lambda row: tag_pos(row["Tokens"]), axis="columns")

In [50]:
display = None
if config["processing"]:
    display = data[["Tokens", "POS"]].head()
display

,Tokens,POS
Id,,
525063,"[want, intent, launched, user, goes, certain, url, example, android, market, http, market, .android, .com, urls, youtube, want, mine, .android, respond, url, intent]","[(want, a), (intent, n), (launched, v), (user, n), (goes, v), (certain, a), (url, a), (example, n), (android, a), (market, n), (http, n), (market, n), (.android, n), (.com, n), (urls, a), (youtube, n), (want, v), (mine, n), (.android, a), (respond, n), (url, a), (intent, n)]"
14128723,"[used, see, building, cleaning, actions, separate, windows, could, wait, complete, continuing, several, weeks, ago, must, clicked, run, background, progress, appears, small, obscured, progress, view, set, progress, things, like, java, build, clean, appear, popup, dialog, eclipse, progress, windows, show]","[(used, v), (see, n), (building, v), (cleaning, v), (actions, n), (separate, a), (windows, n), (could, n), (wait, v), (complete, a), (continuing, v), (several, a), (weeks, n), (ago, r), (must, n), (clicked, v), (run, n), (background, n), (progress, n), (appears, v), (small, a), (obscured, a), (progress, n), (view, n), (set, v), (progress, a), (things, n), (like, n), (java, n), (build, v), (clean, a), (appear, v), (popup, a), (dialog, n), (eclipse, n), (progress, n), (windows, v), (show, n)]"
20728,"[team, develops, distributed, winform, apps, use, clickonce, deployment, pleased, however, found, pain, point, clickonce, creating, deployments, standard, dev, test, production, environments, need, able, create, deployments, install, update, separate, one, another, also, want, control, assemblies, get, deployed, assembly, compiled, mean, want, deployed, obvious, first, choice, creating, deployments, visual, studio, however, vs, really, address, issues, stated, next, line, sdk, tool, mage, mage, works, ok, creating, deployments, rather, tedious, want, every, developer, code, signing, certificate, password, ended, rolling, deployment, app, uses, command, line, version, mage, create, clickonce, manifest, files, satisfied, current, solution, seems, like, would, industry, wide, accepted, approach, problem, best, way, create, ...]","[(team, n), (develops, v), (distributed, v), (winform, n), (apps, n), (use, n), (clickonce, n), (deployment, n), (pleased, v), (however, r), (found, v), (pain, n), (point, n), (clickonce, n), (creating, v), (deployments, n), (standard, a), (dev, a), (test, n), (production, n), (environments, n), (need, v), (able, a), (create, n), (deployments, n), (install, a), (update, a), (separate, a), (one, n), (another, n), (also, r), (want, v), (control, n), (assemblies, n), (get, v), (deployed, v), (assembly, r), (compiled, v), (mean, n), (want, v), (deployed, v), (obvious, a), (first, a), (choice, n), (creating, v), (deployments, n), (visual, a), (studio, n), (however, r), (vs, r), (really, r), (address, a), (issues, n), (stated, v), (next, a), (line, n), (sdk, n), (tool, n), (mage, n), (mage, n), (works, v), (ok, r), (creating, v), (deployments, n), (rather, r), (tedious, a), (want, v), (every, n), (developer, n), (code, n), (signing, v), (certificate, n), (password, n), (ended, v), (rolling, v), (deployment, a), (app, a), (uses, n), (command, v), (line, n), (version, n), (mage, n), (create, n), (clickonce, n), (manifest, a), (files, n), (satisfied, v), (current, a), (solution, n), (seems, v), (like, n), (would, n), (industry, n), (wide, a), (accepted, v), (approach, n), (problem, n), (best, a), (way, n), (create, n), ...]"
6531409,"[ios, app, need, provide, image, filters, based, size, width, height, think, something, similar, large, medium, small, google, images, search, opening, image, reading, dimensions, creating, list, would, performance, intensive, way, get, info, without, opening, image, damien, deville, answered, question, based, suggestion, using, following, code, ios, get, image, dimensions, without, opening]","[(ios, n), (app, v), (need, n), (provide, v), (image, n), (filters, n), (based, v), (size, n), (width, a), (height, n), (thin

***
## 3.5 Lemmatize

In [51]:
if config["processing"]:
    data["Lemmatized"] = data.apply(lambda row: lemmatize(row["POS"], with_pos=True), axis="columns")

In [52]:
display = None
if config["processing"]:
    display = data[["Tokens", "Lemmatized"]].head()
display

,Tokens,Lemmatized
Id,,
525063,"[want, intent, launched, user, goes, certain, url, example, android, market, http, market, .android, .com, urls, youtube, want, mine, .android, respond, url, intent]","[want, intent, launch, user, go, certain, url, example, android, market, http, market, .android, .com, urls, youtube, want, mine, .android, respond, url, intent]"
14128723,"[used, see, building, cleaning, actions, separate, windows, could, wait, complete, continuing, several, weeks, ago, must, clicked, run, background, progress, appears, small, obscured, progress, view, set, progress, things, like, java, build, clean, appear, popup, dialog, eclipse, progress, windows, show]","[use, see, build, clean, action, separate, window, could, wait, complete, continue, several, week, ago, must, click, run, background, progress, appear, small, obscured, progress, view, set, progress, thing, like, java, build, clean, appear, popup, dialog, eclipse, progress, windows, show]"
20728,"[team, develops, distributed, winform, apps, use, clickonce, deployment, pleased, however, found, pain, point, clickonce, creating, deployments, standard, dev, test, production, environments, need, able, create, deployments, install, update, separate, one, another, also, want, control, assemblies, get, deployed, assembly, compiled, mean, want, deployed, obvious, first, choice, creating, deployments, visual, studio, however, vs, really, address, issues, stated, next, line, sdk, tool, mage, mage, works, ok, creating, deployments, rather, tedious, want, every, developer, code, signing, certificate, password, ended, rolling, deployment, app, uses, command, line, version, mage, create, clickonce, manifest, files, satisfied, current, solution, seems, like, would, industry, wide, accepted, approach, problem, best, way, create, ...]","[team, develop, distribute, winform, apps, use, clickonce, deployment, please, however, find, pain, point, clickonce, create, deployment, standard, dev, test, production, environment, need, able, create, deployment, install, update, separate, one, another, also, want, control, assembly, get, deploy, assembly, compile, mean, want, deploy, obvious, first, choice, create, deployment, visual, studio, however, vs, really, address, issue, state, next, line, sdk, tool, mage, mage, work, ok, create, deployment, rather, tedious, want, every, developer, code, sign, certificate, password, end, roll, deployment, app, us, command, line, version, mage, create, clickonce, manifest, file, satisfy, current, solution, seem, like, would, industry, wide, accept, approach, problem, best, way, create, ...]"
6531409,"[ios, app, need, provide, image, filters, based, size, width, height, think, something, similar, large, medium, small, google, images, search, opening, image, reading, dimensions, creating, list, would, performance, intensive, way, get, info, without, opening, image, damien, deville, answered, question, based, suggestion, using, following, code, ios, get, image, dimensions, without, opening]","[io, app, need, provide, image, filter, base, size, width, height, think, something, similar, large, medium, small, google, image, search, open, image, reading, dimension, create, list, would, performance, intensive, way, get, info, without, open, image, damien, deville, answer, question, base, suggestion, use, follow, code, io, get, image, dimension, without, open]"
593205,"[see, line, many, online, examples, using, action, delegate, try, code, get, error, using, generic, type, system, .action, requires, type, arguments, documentation, certainly, describes, form, action, without, type, parameter, missing, action, delegate, zero, parameters]","[see, line, many, online, example, use, action, delegate, try, code, get, error, use, generic, type, system, .action, require, type, argument, documentation, certainly, describe, form, action, without, type, parameter, miss, action, delegate, zero, parameter]"


***
## 3.6 Stemmize

In [53]:
if config["processing"]:
    data["LemmaAndStem"] = data.apply(lambda row: stemmize(row["Tokens"]), axis="columns")

In [54]:
display = None
if config["processing"]:
    display = data[["Tokens", "LemmaAndStem"]].head()
display

,Tokens,LemmaAndStem
Id,,
525063,"[want, intent, launched, user, goes, certain, url, example, android, market, http, market, .android, .com, urls, youtube, want, mine, .android, respond, url, intent]","[want, intent, launch, user, goe, certain, url, exampl, android, market, http, market, .android, .com, url, youtub, want, mine, .android, respond, url, intent]"
14128723,"[used, see, building, cleaning, actions, separate, windows, could, wait, complete, continuing, several, weeks, ago, must, clicked, run, background, progress, appears, small, obscured, progress, view, set, progress, things, like, java, build, clean, appear, popup, dialog, eclipse, progress, windows, show]","[use, see, build, clean, action, separ, window, could, wait, complet, continu, sever, week, ago, must, click, run, background, progress, appear, small, obscur, progress, view, set, progress, thing, like, java, build, clean, appear, popup, dialog, eclips, progress, window, show]"
20728,"[team, develops, distributed, winform, apps, use, clickonce, deployment, pleased, however, found, pain, point, clickonce, creating, deployments, standard, dev, test, production, environments, need, able, create, deployments, install, update, separate, one, another, also, want, control, assemblies, get, deployed, assembly, compiled, mean, want, deployed, obvious, first, choice, creating, deployments, visual, studio, however, vs, really, address, issues, stated, next, line, sdk, tool, mage, mage, works, ok, creating, deployments, rather, tedious, want, every, developer, code, signing, certificate, password, ended, rolling, deployment, app, uses, command, line, version, mage, create, clickonce, manifest, files, satisfied, current, solution, seems, like, would, industry, wide, accepted, approach, problem, best, way, create, ...]","[team, develop, distribut, winform, app, use, clickonc, deploy, pleas, howev, found, pain, point, clickonc, creat, deploy, standard, dev, test, product, environ, need, abl, creat, deploy, instal, updat, separ, one, anoth, also, want, control, assembl, get, deploy, assembl, compil, mean, want, deploy, obvious, first, choic, creat, deploy, visual, studio, howev, vs, realli, address, issu, state, next, line, sdk, tool, mage, mage, work, ok, creat, deploy, rather, tedious, want, everi, develop, code, sign, certif, password, end, roll, deploy, app, use, command, line, version, mage, creat, clickonc, manifest, file, satisfi, current, solut, seem, like, would, industri, wide, accept, approach, problem, best, way, creat, ...]"
6531409,"[ios, app, need, provide, image, filters, based, size, width, height, think, something, similar, large, medium, small, google, images, search, opening, image, reading, dimensions, creating, list, would, performance, intensive, way, get, info, without, opening, image, damien, deville, answered, question, based, suggestion, using, following, code, ios, get, image, dimensions, without, opening]","[io, app, need, provid, imag, filter, base, size, width, height, think, someth, similar, larg, medium, small, googl, imag, search, open, imag, read, dimens, creat, list, would, perform, intens, way, get, info, without, open, imag, damien, devill, answer, question, base, suggest, use, follow, code, io, get, imag, dimens, without, open]"
593205,"[see, line, many, online, examples, using, action, delegate, try, code, get, error, using, generic, type, system, .action, requires, type, arguments, documentation, certainly, describes, form, action, without, type, parameter, missing, action, delegate, zero, parameters]","[see, line, mani, onlin, exampl, use, action, deleg, tri, code, get, error, use, generic, type, system, .action, requir, type, argument, document, certain, describ, form, action, without, type, paramet, miss, action, deleg, zero, paramet]"


***
## 3.7 Generating sentence

In [55]:
if config["processing"]:
    data["Sentence"] = data.apply(lambda row: " ".join([str(item) for item in row["LemmaAndStem"]]), axis="columns")

In [56]:
display = None
if config["processing"]:
    display = data[["LemmaAndStem", "Sentence"]].head()
display

,LemmaAndStem,Sentence
Id,,
525063,"[want, intent, launch, user, goe, certain, url, exampl, android, market, http, market, .android, .com, url, youtub, want, mine, .android, respond, url, intent]",want intent launch user goe certain url exampl android market http market .android .com url youtub want mine .android respond url intent
14128723,"[use, see, build, clean, action, separ, window, could, wait, complet, continu, sever, week, ago, must, click, run, background, progress, appear, small, obscur, progress, view, set, progress, thing, like, java, build, clean, appear, popup, dialog, eclips, progress, window, show]",use see build clean action separ window could wait complet continu sever week ago must click run background progress appear small obscur progress view set progress thing like java build clean appear popup dialog eclips progress window show
20728,"[team, develop, distribut, winform, app, use, clickonc, deploy, pleas, howev, found, pain, point, clickonc, creat, deploy, standard, dev, test, product, environ, need, abl, creat, deploy, instal, updat, separ, one, anoth, also, want, control, assembl, get, deploy, assembl, compil, mean, want, deploy, obvious, first, choic, creat, deploy, visual, studio, howev, vs, realli, address, issu, state, next, line, sdk, tool, mage, mage, work, ok, creat, deploy, rather, tedious, want, everi, develop, code, sign, certif, password, end, roll, deploy, app, use, command, line, version, mage, creat, clickonc, manifest, file, satisfi, current, solut, seem, like, would, industri, wide, accept, approach, problem, best, way, creat, ...]",team develop distribut winform app use clickonc deploy pleas howev found pain point clickonc creat deploy standard dev test product environ need abl creat deploy instal updat separ one anoth also want control assembl get deploy assembl compil mean want deploy obvious first choic creat deploy visual studio howev vs realli address issu state next line sdk tool mage mage work ok creat deploy rather tedious want everi develop code sign certif password end roll deploy app use command line version mage creat clickonc manifest file satisfi current solut seem like would industri wide accept approach problem best way creat clickonc deploy
6531409,"[io, app, need, provid, imag, filter, base, size, width, height, think, someth, similar, larg, medium, small, googl, imag, search, open, imag, read, dimens, creat, list, would, perform, intens, way, get, info, without, open, imag, damien, devill, answer, question, base, suggest, use, follow, code, io, get, imag, dimens, without, open]",io app need provid imag filter base size width height think someth similar larg medium small googl imag search open imag read dimens creat list would perform intens way get info without open imag damien devill answer question base suggest use follow code io get imag dimens without open
593205,"[see, line, mani, onlin, exampl, use, action, deleg, tri, code, get, error, use, generic, type, system, .action, requir, type, argument, document, certain, describ, form, action, without, type, paramet, miss, action, deleg, zero, paramet]",see line mani onlin exampl use action deleg tri code get error use generic type system .action requir type argument document certain describ form action without type paramet miss action deleg zero paramet


***
## 3.8 Saving

In [57]:
display = None
if config["processing"]:
    display = data.head()
display

,Title,Body,Tags,Tokens,POS,Lemmatized,LemmaAndStem,Sentence
Id,,,,,,,,
525063,android respond to url in intent,i want my intent to be launched when the user goes to a certain url: for example the android market does this with http://market.android.com/ urls. so does youtube. i want mine to do that too.,"['android', 'url', 'android-intent', 'intentfilter', 'launch']","[want, intent, launched, user, goes, certain, url, example, android, market, http, market, .android, .com, urls, youtube, want, mine, .android, respond, url, intent]","[(want, a), (intent, n), (launched, v), (user, n), (goes, v), (certain, a), (url, a), (example, n), (android, a), (market, n), (http, n), (market, n), (.android, n), (.com, n), (urls, a), (youtube, n), (want, v), (mine, n), (.android, a), (respond, n), (url, a), (intent, n)]","[want, intent, launch, user, go, certain, url, example, android, market, http, market, .android, .com, urls, youtube, want, mine, .android, respond, url, intent]","[want, intent, launch, user, goe, certain, url, exampl, android, market, http, market, .android, .com, url, youtub, want, mine, .android, respond, url, intent]",want intent launch user goe certain url exampl android market http market .android .com url youtub want mine .android respond url intent
14128723,eclipse -- progress windows don't show up any more,i used to see both building and cleaning actions in their own separate windows where i could wait for them to complete before continuing. several weeks ago i must have clicked run in background because now the progress only appears in a small obscured progress view. how do i set the progress of things like java build and clean to appear in a popup dialog again?,"['java', 'eclipse']","[used, see, building, cleaning, actions, separate, windows, could, wait, complete, continuing, several, weeks, ago, must, clicked, run, background, progress, appears, small, obscured, progress, view, set, progress, things, like, java, build, clean, appear, popup, dialog, eclipse, progress, windows, show]","[(used, v), (see, n), (building, v), (cleaning, v), (actions, n), (separate, a), (windows, n), (could, n), (wait, v), (complete, a), (continuing, v), (several, a), (weeks, n), (ago, r), (must, n), (clicked, v), (run, n), (background, n), (progress, n), (appears, v), (small, a), (obscured, a), (progress, n), (view, n), (set, v), (progress, a), (things, n), (like, n), (java, n), (build, v), (clean, a), (appear, v), (popup, a), (dialog, n), (eclipse, n), (progress, n), (windows, v), (show, n)]","[use, see, build, clean, action, separate, window, could, wait, complete, continue, several, week, ago, must, click, run, background, progress, appear, small, obscured, progress, view, set, progress, thing, like, java, build, clean, appear, popup, dialog, eclipse, progress, windows, show]","[use, see, build, clean, action, separ, window, could, wait, complet, continu, sever, week, ago, must, click, run, background, progress, appear, small, obscur, progress, view, set, progress, thing, like, java, build, clean, appear, popup, dialog, eclips, progress, window, show]",use see build clean action separ window could wait complet continu sever week ago must click run background progress appear small obscur progress view set progress thing like java build clean appear popup dialog eclips progress window show
20728,what's the best way to create clickonce deployments,our team develops distributed winform apps. we use clickonce for deployment and are very pleased with it. however we've found the pain point with clickonce is in creating the deployments. we have the standard dev/test/production environments and need to be able to create deployments for each of these that install and update separate from one another. also we want control over what assemblies get deployed. just because an assembly was compiled doesn't mean we want it deployed. the obvious first choice for creating deployments is visual studio. however vs really doesn't address the issues stat

In [58]:
if config["processing"]:
    data.to_csv("data/data_cleaned.csv", index_label="Id")